In [2]:
import numpy as np
from typing import Union, Callable
import pytest
import ipytest
from copy import deepcopy

from forward_propagation import (
    forward_propagation, 
    create_weights, 
    vector_to_weights)

from activation_functions import relu, sigmoid 
import test_functions
from gradient_descent import gradient_descent

#%load_ext autoreload
%autoreload 2
np.random.seed(42)

UsageError: Line magic function `%autoreload` not found.


## Multiple inputs 

In [3]:
inputs = np.array([[1,2,5,4]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

array([[0.99899323, 1.        , 0.99945816]])

In [4]:
inputs = np.array([[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

array([[0.53469416, 0.94103299, 0.79062883]])

In [5]:
inputs = np.array([[1,2,5,4],[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

array([[0.99899323, 1.        , 0.99945816],
       [0.53469416, 0.94103299, 0.79062883]])

In [6]:
inputs = np.array([[1,2,5,4],[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        ),
    np.array(
            [
                [1,0.2,0.5,1],
                [2,1,3,5]
            ]
        )
    ]
activation = sigmoid
forward_propagation(inputs,weights,activation)

array([[0.93695121, 0.99998324],
       [0.89266103, 0.99991581]])

### Creating a data set 
Create a data set of points sampled randomly from a function.

In [42]:
# very easy function
def linear_function(x):
    d=len(x)
    xstar=np.array(range(1,(d+1)))
    xx=x.dot(xstar) + 3
    return xx

In [43]:
def simulate_data_target(fun: Callable,
                       n_features: int,
                       n_obs: int,
                       LB: list[float] = -5,
                       UB: list[float] = 5) -> dict:
    
    entry_data = np.random.uniform(low= [LB] * n_features,
                                   high= [UB] * n_features,
                                   size=(n_obs, n_features))
    target = np.apply_along_axis(fun, 1, entry_data)
    
    return {"data": entry_data, "target": target}

In [44]:
#used_function = test_functions.sphere
used_function = linear_function
simulated_data = simulate_data_target(fun = used_function,n_features = 2,n_obs=10)

In [45]:
simulated_data

{'data': array([[-3.4398136 , -3.4400548 ],
        [-4.41916388,  3.66176146],
        [ 1.01115012,  2.08072578],
        [-4.79415506,  4.69909852],
        [ 3.32442641, -2.87660889],
        [-3.18175033, -3.1659549 ],
        [-1.95757757,  0.24756432],
        [-0.68054981, -2.0877086 ],
        [ 1.11852895, -3.60506139],
        [-2.07855351, -1.33638157]]),
 'target': array([-7.31992319,  5.90435904,  8.17260167,  7.60404199,  0.57120862,
        -6.51366013,  1.53755106, -1.85596701, -3.09159384, -1.75131665])}

In [46]:
-3.4398136 + 2*-3.4400548 

-10.3199232

### Neural Network weight and output

In [47]:
weights = create_weights([2,1])
predicted_output = forward_propagation(simulated_data["data"],weights,sigmoid)

In [48]:
weights

[array([[ 0.28677805, -0.07982693,  0.37394315]])]

In [49]:
simulated_data["target"]

array([-7.31992319,  5.90435904,  8.17260167,  7.60404199,  0.57120862,
       -6.51366013,  1.53755106, -1.85596701, -3.09159384, -1.75131665])

In [50]:
predicted_output.reshape(-1,)

array([0.41631704, 0.23403188, 0.62194709, 0.20164737, 0.82591476,
       0.42903619, 0.4483822 , 0.58550936, 0.72760274, 0.47116674])

In [51]:
vector_to_weights([0.28677805, -0.07982693,  0.37394315],[2,1])

[array([[ 0.28677805, -0.07982693,  0.37394315]])]

### Error function

In [52]:
# mean squared error
def cost_function_mse(y_predicted: np.ndarray,y_observed: np.ndarray):
    error = np.mean((y_predicted - y_observed)**2)
    return error

In [53]:
# entropy
def cost_function_entropy(y_predicted: np.ndarray,y_observed: np.ndarray):

    n = len(y_observed)
    
    term_A = np.multiply(np.log(y_predicted),y_observed)
    term_B = np.multiply(1-y_observed,np.log(1-y_predicted))
    
    error = - (1/n)*(np.sum(term_A)+np.sum(term_B))

    return(error)

In [54]:
def error_with_parameters(vector_weights: np.ndarray,
                          activation_function: Callable,
                          data: dict,
                          cost_function: Callable) -> float:
    
    weights = vector_to_weights(vector_weights,used_network_structure)
    predicted_output = forward_propagation(data["data"],weights,activation_function)
    predicted_output = predicted_output.reshape(-1,)
    
    error = cost_function(predicted_output,data["target"]) + regularization * np.sum(np.abs(vector_weights))
    
    return error

In [55]:
used_network_structure = [2,1] # 2 inputs features, 1 layer with 1 node
used_activation = relu
used_data = simulated_data
used_cost_function = cost_function_mse

def neural_network_cost(vector_weights):
    
    cost = error_with_parameters(vector_weights,
                          activation_function = used_activation,
                          data = used_data,
                          cost_function = used_cost_function)
    
    return cost

In [59]:
np.sum(np.abs(np.array([0.28677805, -0.07982693,  0.37394315])))

0.7405481300000001

In [56]:
neural_network_cost(np.array([0.28677805, -0.07982693,  0.37394315]))

27.54414127554354

In [57]:
gradient_descent(func = neural_network_cost,
                 start_x = np.array([0.28677805, -0.07982693,  0.37394315]),
                 LB = [-5] * 3, UB = [5] * 3,budget = 1000000)

{'time_used': 1496,
 'x_best': array([0.99999995, 1.99999991, 2.99999985]),
 'f_best': 11.207871980599968,
 'hist_f_best': [27.54414127554354,
  20.14932482085556,
  19.949697018510847,
  14.1538146883292,
  12.681703712434329,
  11.558587532147481,
  11.553882356939372,
  11.54649684285076,
  11.540896969388161,
  11.53357502307076,
  11.526845346956655,
  11.521741399945364,
  11.521489249063778,
  11.51989168847274,
  11.515338629993357,
  11.5115855553471,
  11.510118879819625,
  11.509870903616788,
  11.509789688474578,
  11.507042787127649,
  11.504513390800456,
  11.50298561727374,
  11.502259671856155,
  11.501529832119486,
  11.500653606953888,
  11.50041119781182,
  11.498974949033515,
  11.497714099134663,
  11.49684104332641,
  11.496052681849985,
  11.49567294381592,
  11.495175980895606,
  11.494527726877061,
  11.494272691574938,
  11.493555222975502,
  11.493426575918296,
  11.492765438253237,
  11.492611673819631,
  11.492049460439748,
  11.491780804152285,
  11.491335